<a href="https://colab.research.google.com/github/RohitKumar2705/GANS/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Flatten ,Dropout,Reshape,Activation ,BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Conv2D,Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
img_rows = 28
img_cols = 28
channels = 1

# input image dimensions
img_shape = (img_rows,img_cols,channels)

z_dim = 100

In [3]:
def build_generator(z_dim):
  model = Sequential()

  model.add(Dense(256*7**7,input_dim = z_dim))
  model.add(Reshape(7,7,256))

  # transpose convolutional
  model.add(Conv2DTranspose(128,kernel_size = 3,strides = 2, padding = 'same'))

  model.add(BatchNormalization())

  model.add(LeakyReLU(alpha = 0.01))

  model.add(Conv2DTranspose(64,kernel_size = 4,strides = 1,padding = 'same'))

  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))

    # Transposed convolution layer, from 14x14x64 to 28x28x1 tensor
  model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))

    # Output layer with tanh activation
  model.add(Activation('tanh'))

  return model


In [4]:
def build_discriminator(img_shape):
  model = Sequential()

  model.add(Conv2D(32,kernel_size = 3,
                   strides = 2,
                   input_shape = img_shape,
                   padding = 'same'))
  # leaky relu activation
  model.add(BatchNormalization())

  # leaky leaky relu activation
  model.add(LeakyReLU(alpha = 0.01))

  # Convolutional layer, from 7x7x64 tensor into 3x3x128 tensor
  model.add(
        Conv2D(128,
               kernel_size=3,
               strides=2,
               input_shape=img_shape,
               padding='same'))

    # Batch normalization
  model.add(BatchNormalization())

    # Leaky ReLU activation
  model.add(LeakyReLU(alpha=0.01))

    # Output layer with sigmoid activation
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  return model

In [5]:
def build_gan(generator,discriminator):
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  return model

In [ ]:
# build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss = 'binary_crossentropy',optimizer = Adam(),metrics = ['accuracy'])

# build the generator
generator = build_generator(z_dim)

discriminator.trainable = False

# build gan
gan = build_gan(generator,discriminator)
gan.compile(loss = 'binary_crossentropy',optimizer = Adam())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
losses = []
accurices = []
iteration_checkpoints = []

def train(iterations,batch_size,sample_interval):
  (X_train,_),(_,_) =  mnist.load_data()

  X_train = X_train/127.5 - 1.0
  X_train = np.expand_dims(X_train,axis = 3)
  real = np.ones((batch_size , 1))
  fake = np.zeros((batch_size,1))

  for iteration in range(iterations):
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    # generate a batch of fake image
    z = np.random.normal(0,1,(batch_size,100))
    gen_imgs = generator.predict(z)

    # train discriminator
    d_loss_real = discriminator.train_on_loss(imgs,real)
    d_loss_fake = discriminator.train_on_loss(gen_imgs,fake)
    d_loss,accuracy = 0.5*np.add(d_loss_real,d_loss_fake)
    # Generate a batch of fake images
    z = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict(z)

        # Train Generator
    g_loss = gan.train_on_batch(z, real)

    if (iteration + 1) % sample_interval == 0:

            # Save losses and accuracies so they can be plotted after training
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # Output training progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

            # Output a sample of generated image
            sample_images(generator)

In [ ]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):

    # Sample random noise
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # Generate images from random noise
    gen_imgs = generator.predict(z)

    # Rescale image pixel values to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Set image grid
    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # Output a grid of images
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1

In [ ]:
# Set hyperparameters
iterations = 5000
batch_size = 128
sample_interval = 1000

# Train the DCGAN for the specified number of iterations
train(iterations, batch_size, sample_interval)

In [ ]:

losses = np.array(losses)

# Plot training losses for Discriminator and Generator
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, losses.T[0], label="Discriminator loss")
plt.plot(iteration_checkpoints, losses.T[1], label="Generator loss")

plt.xticks(iteration_checkpoints, rotation=90)

plt.title("Training Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()

In [ ]:
accuracies = np.array(accuracies)

# Plot Discriminator accuracy
plt.figure(figsize=(15, 5))
plt.plot(iteration_checkpoints, accuracies, label="Discriminator accuracy")

plt.xticks(iteration_checkpoints, rotation=90)
plt.yticks(range(0, 100, 5))

plt.title("Discriminator Accuracy")
plt.xlabel("Iteration")
plt.ylabel("Accuracy (%)")
plt.legend()